In [1]:
import flwr as fl
from pathlib import Path
import os
import tensorflow as tf

from sources.simulation_framework.base_simulator import BaseSimulator
from sources.datasets.femnist.femnist_client_dataset_factory import FemnistClientDatasetFactory
from sources.flwr_parameters.default_parameters import DEFAULT_SEED
from sources.models.femnist.femnist_model_template import FemnistModelTemplate
from sources.flwr_strategies.model_logging_strategy_decorator import ModelLoggingStrategyDecorator
from sources.flwr_strategies.evaluation_metrics_logging_strategy_decorator import \
    EvaluationMetricsLoggingStrategyDecorator
%load_ext autoreload
%autoreload 2

2021-12-05 13:26:26.655354: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2021-12-05 13:26:26.655372: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
base_dir = Path(os.getcwd()).parent.parent / "data"
experiment_name = "initial_experiment"
model_saving_dir = str(base_dir / experiment_name / "models")
metrics_saving_dir = str(base_dir / experiment_name / "metrics")

In [3]:
def eval_config(rnd: int):
    """Return training configuration dict for each round.
    Keep batch size fixed at 32, perform two rounds of training with one
    local epoch, increase to two local epochs afterwards.
    """
    config = {
        "batch_size": 32,
        "val_steps": 2
    }
    return config

def fit_config(rnd: int):
    """Return training configuration dict for each round.
    Keep batch size fixed at 32, perform two rounds of training with one
    local epoch, increase to two local epochs afterwards.
    """
    config = {
        "batch_size": 32,
        "local_epochs": 1 if rnd < 2 else 2,
    }
    return config

simulation_parameters = {"num_rounds": 2, "num_clients": 10}
strategy = fl.server.strategy.FedAvg(on_fit_config_fn=fit_config, on_evaluate_config_fn=eval_config)

model_template = FemnistModelTemplate(DEFAULT_SEED)
dataset_factory = FemnistClientDatasetFactory(base_dir)

simulator = BaseSimulator(simulation_parameters, strategy, model_template, dataset_factory)

In [4]:
simulator.start_simulation()




INFO flower 2021-12-05 13:26:28,557 | app.py:77 | Flower server running (insecure, 2 rounds)
INFO flower 2021-12-05 13:26:28,559 | server.py:118 | Initializing global parameters
INFO flower 2021-12-05 13:26:28,561 | server.py:304 | Requesting initial parameters from one random client
2021-12-05 13:26:30.597864: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-05 13:26:30.598366: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2021-12-05 13:26:30.599495: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2

1/1 [==============================] - ETA: 0s - loss: 38.4232 - acc: 0.0000e+00

1/3 [=========>....................] - ETA: 1s - loss: 48.9415 - acc: 0.0000e+00

3/3 [==============================] - ETA: 0s - loss: 7660990.0000 - acc: 0.0294

1/1 [==============================] - 1s 745ms/step - loss: 38.4232 - acc: 0.0000e+00 - val_loss: 78940.0234 - val_acc: 0.1333


...

1/1 [==============================] - 0s 92ms/step - loss: 1453491960676352.0000 - acc: 0.0000e+00 - val_loss: 1952157099904997305352192.0000 - val_acc: 0.0000e+00


DEBUG flower 2021-12-05 13:26:33,910 | server.py:260 | fit_round received 2 results and 0 failures
DEBUG flower 2021-12-05 13:26:33,944 | server.py:201 | evaluate_round: strategy sampled 2 clients (out of 10)


2/2 [==============================] - ETA: 0s - loss: 114966945531592274083840.0000 - acc: 0.1667 - ETA: 0s - loss: 121966431065252244226048.0000 - acc: 0.1250 - 0s 7ms/step - loss: 125135965403803797159936.0000 - acc: 0.1212


2/2 [==============================] - 1s 492ms/step - loss: 114966945531592274083840.0000 - acc: 0.1667


DEBUG flower 2021-12-05 13:26:34,612 | server.py:210 | evaluate_round received 2 results and 0 failures
INFO flower 2021-12-05 13:26:34,613 | server.py:172 | FL finished in 3.596156666999832
INFO flower 2021-12-05 13:26:34,615 | app.py:119 | app_fit: losses_distributed [(1, 147849.65274234695), (2, 1.2029357498846497e+23)]
INFO flower 2021-12-05 13:26:34,616 | app.py:120 | app_fit: metrics_distributed {}
INFO flower 2021-12-05 13:26:34,617 | app.py:121 | app_fit: losses_centralized []
INFO flower 2021-12-05 13:26:34,618 | app.py:122 | app_fit: metrics_centralized {}
DEBUG flower 2021-12-05 13:26:34,633 | connection.py:68 | Insecure gRPC channel closed
DEBUG flower 2021-12-05 13:26:34,633 | connection.py:68 | Insecure gRPC channel closed
DEBUG flower 2021-12-05 13:26:34,633 | connection.py:68 | Insecure gRPC channel closed
DEBUG flower 2021-12-05 13:26:34,633 | connection.py:68 | Insecure gRPC channel closed
DEBUG flower 2021-12-05 13:26:34,635 | connection.py:68 | Insecure gRPC channel